In [ ]:
import requests
import json
import pandas
import numpy as np
import matplotlib.pyplot as plt

### 공동주거 데이터 주소 수집

In [68]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from requests.utils import unquote

def get_data_building(kapt_codes):
    service_key = unquote("iCNxo2r0TdZnnV63%2FItO%2BQrOUqJakXCxx%2Fm20BsCp53DGZzJMDd1%2F7jOGLYQE%2BSn%2B1EQeSeIhUsTIyQ5dYgy4Q%3D%3D")
    url = "https://apis.data.go.kr/1613000/AptBasisInfoServiceV2/getAphusBassInfoV2"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
        "Accept": "application/xml"
    }
    all_data = []
    missing_codes = []
    
    for kapt_code in kapt_codes:  # ✅ 반복문 안에서 요청이 올바르게 수행되도록 수정
        params = {
            "serviceKey": service_key,
            "kaptCode": kapt_code,
            "_type": "xml"
        }

        response = requests.get(url, params=params, headers=headers)
        print("데이터 수집을 시작합니다.")
        
        if response.status_code == 200:
            root = ET.fromstring(response.text)
            items = root.findall(".//item")
            
            if not items:
                missing_codes.append(kapt_code)  # 코드가 없으면 추적 리스트에 추가
            
            for item in items:
                data_dict = {child.tag: child.text if child.text else 'N/A' for child in item}
                all_data.append(data_dict)
        else:
            missing_codes.append(kapt_code)  # 요청 실패 시 코드 추가

    # ✅ DataFrame 생성 (결과가 없을 경우 빈 DataFrame 반환)
    df = pd.DataFrame(all_data) if all_data else pd.DataFrame()
    
    if missing_codes:
        print(f"다음 kaptCode에 대한 데이터가 존재하지 않습니다: {missing_codes}")

    return df

### 시도코드를 이용한 데이터 수집

In [ ]:
def get_kapt_codes(sido_code):
    
    service_key = unquote("iCNxo2r0TdZnnV63%2FItO%2BQrOUqJakXCxx%2Fm20BsCp53DGZzJMDd1%2F7jOGLYQE%2BSn%2B1EQeSeIhUsTIyQ5dYgy4Q%3D%3D")

    url = "https://apis.data.go.kr/1613000/AptListService2/getSidoAptList"

    params = {
        "serviceKey": service_key,
        "sidoCode": sido_code,  # 시도코드
        "pageNo": "1",  # 페이지번호
        "numOfRows": "10000",  # 목록 개수
        "_type": "xml"
    }

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
        "Accept": "application/xml"
    }

    # ✅ API 요청 보내기 (GET 방식)
    response = requests.get(url, params=params, headers=headers)

    # ✅ 응답 확인 및 XML 파싱 및 DataFrame 변환
    if response.status_code == 200:
        root = ET.fromstring(response.text)
        data_list = []
        for item in root.findall(".//item"):
            data_dict = {child.tag: child.text if child.text else 'N/A' for child in item}
            data_list.append(data_dict)
    
        # DataFrame 생성
        
    df = pd.DataFrame(data_list)
        
    return df

In [ ]:
# sido_cod = [11, 21,22,23,24,25,26,29,31,32,33,34,35,36,37,38,39]
seoul_sample = get_kapt_codes(11)
busan_sample = get_kapt_codes(21)
daegu_sample = get_kapt_codes(22)
incheon_sample = get_kapt_codes(23)
gwangju_sample = get_kapt_codes(24)
dageon_sample = get_kapt_codes(25)
yulsan_sample = get_kapt_codes(26)
sejong_sample = get_kapt_codes(29)
gyonggi_sample = get_kapt_codes(31)
gangwon_sample = get_kapt_codes(32)
chungbuk_sample = get_kapt_codes(33)
chungnam_sample = get_kapt_codes(34)
jeonbuk_sample = get_kapt_codes(35)
jeonnam_sample = get_kapt_codes(36)
gyeongbuk_sample = get_kapt_codes(37)
gyeongnam_sample = get_kapt_codes(38)
jeju_sample = get_kapt_codes(39)

In [89]:
# 공동주거 데이터의 정보 파일
seoul_data = get_data_building(seoul_sample["kaptCode"].tolist())

In [113]:
seoul_data.to_csv("seoul_data.csv", index=False)

### 전력사용량 데이터 수집 -> 전력사용량 코드 손 봐야 함.

In [114]:
def get_energyconsumption(kapt_codes, date):
    service_key = unquote("iCNxo2r0TdZnnV63%2FItO%2BQrOUqJakXCxx%2Fm20BsCp53DGZzJMDd1%2F7jOGLYQE%2BSn%2B1EQeSeIhUsTIyQ5dYgy4Q%3D%3D")
    url = "http://apis.data.go.kr/1611000/ApHusEnergyUseInfoOfferService"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
        "Accept": "application/xml"
    }
    all_data = []
    missing_codes = []
    
    for kapt_code, date in zip(kapt_codes, date):  # 리스트로 변경후 값을 넣어야 한다.

        params = {
            "serviceKey": service_key,
            "kaptCode": kapt_code,
            "_type": "xml",
            "reqDate" : date 
        }

        response = requests.get(url, params=params, headers=headers)
        print("데이터 수집을 시작합니다.")
        
        if response.status_code == 200:
            root = ET.fromstring(response.text)
            items = root.findall(".//item")
            
            if not items:
                missing_codes.append(kapt_code)  # 코드가 없으면 추적 리스트에 추가
            
            for item in items:
                data_dict = {child.tag: child.text if child.text else 'N/A' for child in item}
                all_data.append(data_dict)
        else:
            missing_codes.append(kapt_code)  # 요청 실패 시 코드 추가

    # ✅ DataFrame 생성 (결과가 없을 경우 빈 DataFrame 반환)
    df = pd.DataFrame(all_data) if all_data else pd.DataFrame()
    
    if missing_codes:
        print(f"다음 kaptCode에 대한 데이터가 존재하지 않습니다: {missing_codes}")

    return df

In [122]:
seoul_data.rename(columns = {'doroJuso' : 'address'}, inplace=True)

### 지도 그리는 함수 생성

In [143]:
import requests
import pandas as pd

KAKAO_API_KEY = "b83a0138ed0d17b68f0c86fe6808b74d"

def get_lat_lon(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"

    headers = {
        "Authorization": f"KakaoAK {KAKAO_API_KEY}",
        "KA": "python/3.9.0 myAppName",  # KA 헤더 추가 (Python 버전과 앱 이름 설정)
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"  # User-Agent 추가
    }

    params = {"query": address}

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        result = response.json()
        if result["documents"]:
            lat = float(result["documents"][0]["y"])
            lon = float(result["documents"][0]["x"])
            return lat, lon
        else:
            print(f"주소 변환 실패: {address}")  # 변환되지 않은 주소 출력
    else:
        print(f"API 요청 실패: {response.status_code}, {response.text}")  # 오류 메시지 출력

    return None, None

# 🔹 CSV 파일에서 주소 데이터 읽기
df["lat"], df["lon"] = zip(*df["address"].apply(get_lat_lon))

# 🔹 HTML 파일 생성
html_template = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8"/>
    <title>Kakao 지도 - Python 마커 표시</title>
    <script type="text/javascript" src="//dapi.kakao.com/v2/maps/sdk.js?appkey=41c870c490aebd3810bd1152b41c792e"></script>
</head>
<body>
    <div id="map" style="width:800px;height:600px;"></div>
    <script>
        var map = new kakao.maps.Map(document.getElementById('map'), {{
            center: new kakao.maps.LatLng({df['lat'].mean()}, {df['lon'].mean()}), 
            level: 6 
        }});

        var positions = {df.to_json(orient='records')};

        positions.forEach(function(position) {{
            var marker = new kakao.maps.Marker({{
                map: map,
                position: new kakao.maps.LatLng(position.lat, position.lon)
            }});
            
            var infowindow = new kakao.maps.InfoWindow({{
                content: '<div style="padding:5px;">' + position.name + '</div>'
            }});

            kakao.maps.event.addListener(marker, 'click', function() {{
                infowindow.open(map, marker);
            }});
        }});
    </script>
</body>
</html>
"""

# 🔹 HTML 파일 저장
with open("map.html", "w", encoding="utf-8") as f:
    f.write(html_template)

print("HTML 파일이 생성되었습니다. 'python3 -m http.server 8080'을 실행하고 http://localhost:8080/map.html 에 접속하세요!")

HTML 파일이 생성되었습니다. 'python3 -m http.server 8080'을 실행하고 http://localhost:8080/map.html 에 접속하세요!
